## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
nat_gas = pd.read_csv('/content/drive/MyDrive/JP & Chase Analytics/Nat_Gas.csv')
nat_gas.head()

,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


## Prepare Data

### Subtask:
Ensure the 'Dates' column in the `nat_gas` DataFrame is converted to datetime objects. Although the `nat_gas` DataFrame's prices aren't directly used by the pricing function (as prices are passed as inputs), this is a good preparatory step for any time-series analysis.


**Reasoning**:
To ensure the 'Dates' column is in the correct format for time-series analysis, I will convert it to datetime objects using `pd.to_datetime()` with the specified format.



In [3]:
nat_gas['Dates'] = pd.to_datetime(nat_gas['Dates'], format='%m/%d/%y')
print("Converted 'Dates' column to datetime objects.")
nat_gas.info()

Converted 'Dates' column to datetime objects.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Dates   48 non-null     datetime64[ns]
 1   Prices  48 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 900.0 bytes


## Define Pricing Function




**Reasoning**:
I need to define the `price_contract` function as specified, handling event processing, storage level tracking, rate limits, capacity limits, and cost calculations. This involves combining, sorting events, iterating through them, and updating the total value based on injections, withdrawals, and storage costs.



In [4]:
import datetime

def price_contract(injection_events, withdrawal_events, injection_rate, withdrawal_rate, max_volume, storage_cost_per_unit_per_day):
    """
    Calculates the net value of a natural gas storage contract.

    Args:
        injection_events (list): List of tuples (date, volume, price) for injections.
        withdrawal_events (list): List of tuples (date, volume, price) for withdrawals.
        injection_rate (float): Maximum volume that can be injected per event.
        withdrawal_rate (float): Maximum volume that can be withdrawn per event.
        max_volume (float): Maximum storage capacity.
        storage_cost_per_unit_per_day (float): Daily cost per unit of stored gas.

    Returns:
        float: The net value of the contract.
    """
    storage_level = 0.0
    total_value = 0.0
    last_event_date = None

    all_events = []
    for date, volume, price in injection_events:
        all_events.append((date, volume, price, 'injection'))
    for date, volume, price in withdrawal_events:
        all_events.append((date, volume, price, 'withdrawal'))

    all_events.sort(key=lambda x: x[0]) # Sort by date

    for event_date, event_volume, event_price, event_type in all_events:
        if last_event_date is None:
            last_event_date = event_date

        # Calculate storage cost for the period between events
        if event_date > last_event_date:
            days_passed = (event_date - last_event_date).days
            if days_passed > 0:
                storage_cost = days_passed * storage_level * storage_cost_per_unit_per_day
                total_value -= storage_cost

        last_event_date = event_date

        if event_type == 'injection':
            # Apply injection rate limit
            actual_injection_volume = min(event_volume, injection_rate)
            # Apply capacity limit
            actual_injection_volume = min(actual_injection_volume, max_volume - storage_level)

            if actual_injection_volume > 0:
                total_value -= actual_injection_volume * event_price # Cost for injection
                storage_level += actual_injection_volume

        elif event_type == 'withdrawal':
            # Apply withdrawal rate limit
            actual_withdrawal_volume = min(event_volume, withdrawal_rate)
            # Apply available storage limit
            actual_withdrawal_volume = min(actual_withdrawal_volume, storage_level)

            if actual_withdrawal_volume > 0:
                total_value += actual_withdrawal_volume * event_price # Revenue from withdrawal
                storage_level -= actual_withdrawal_volume

        # Ensure storage_level stays within bounds (should be handled by min() logic above, but as a safeguard)
        storage_level = max(0.0, min(storage_level, max_volume))

    return total_value

print("price_contract function defined.")

price_contract function defined.


## Generate Sample Inputs




**Reasoning**:
I need to define parameters and create sample injection and withdrawal events for two distinct scenarios to test the `price_contract` function, as instructed by the subtask.



In [5]:
import datetime

# Scenario 1: Basic operations with sufficient capacity and rates
print("--- Scenario 1: Basic Operations ---")
injection_rate_1 = 500.0
withdrawal_rate_1 = 600.0
max_volume_1 = 1000.0
storage_cost_per_unit_per_day_1 = 0.01

injection_events_1 = [
    (datetime.date(2023, 1, 1), 300, 5.0),  # Inject 300 units at $5
    (datetime.date(2023, 1, 10), 400, 5.5), # Inject 400 units at $5.5
    (datetime.date(2023, 2, 1), 200, 6.0)   # Inject 200 units at $6
]

withdrawal_events_1 = [
    (datetime.date(2023, 1, 15), 200, 7.0), # Withdraw 200 units at $7
    (datetime.date(2023, 2, 10), 350, 7.5)  # Withdraw 350 units at $7.5
]

print("Scenario 1 parameters and events defined.")

# Scenario 2: Testing capacity limits, rate limits, and longer storage periods
print("\n--- Scenario 2: Capacity & Rate Limits, Longer Storage ---")
injection_rate_2 = 200.0  # Lower injection rate
withdrawal_rate_2 = 250.0 # Lower withdrawal rate
max_volume_2 = 500.0      # Smaller max capacity
storage_cost_per_unit_per_day_2 = 0.02 # Higher storage cost

injection_events_2 = [
    (datetime.date(2023, 3, 1), 300, 4.0),  # Attempt to inject more than max_volume and injection_rate
    (datetime.date(2023, 3, 15), 100, 4.2), # Inject more
    (datetime.date(2023, 4, 1), 250, 4.5)   # Another injection, potentially hitting capacity
]

withdrawal_events_2 = [
    (datetime.date(2023, 3, 20), 400, 6.0), # Attempt to withdraw more than available and withdrawal_rate
    (datetime.date(2023, 4, 10), 150, 6.5), # Withdraw more
    (datetime.date(2023, 5, 1), 100, 7.0)   # Final withdrawal after longer storage
]

print("Scenario 2 parameters and events defined.")

--- Scenario 1: Basic Operations ---
Scenario 1 parameters and events defined.

--- Scenario 2: Capacity & Rate Limits, Longer Storage ---
Scenario 2 parameters and events defined.


## Test `price_contract` Function

### Subtask:
Test the `price_contract` function with the previously generated sample inputs for Scenario 1 and Scenario 2. Print the net value calculated for each scenario.



**Reasoning**:
I need to call the `price_contract` function with the parameters and event lists defined for Scenario 1 and Scenario 2, then print the calculated net value for each to test the function as per the subtask.



In [6]:
print("\n--- Testing Scenario 1 ---")
net_value_1 = price_contract(injection_events_1, withdrawal_events_1, injection_rate_1, withdrawal_rate_1, max_volume_1, storage_cost_per_unit_per_day_1)
print(f"Net value for Scenario 1: ${net_value_1:.2f}")

print("\n--- Testing Scenario 2 ---")
net_value_2 = price_contract(injection_events_2, withdrawal_events_2, injection_rate_2, withdrawal_rate_2, max_volume_2, storage_cost_per_unit_per_day_2)
print(f"Net value for Scenario 2: ${net_value_2:.2f}")


--- Testing Scenario 1 ---
Net value for Scenario 1: $-1085.00

--- Testing Scenario 2 ---
Net value for Scenario 2: $870.00


## Summarize Functionality and Test Results



### Reasoning:
To complete the task, I need to provide a clear summary of what the `price_contract` function does, how it handles various conditions (like rate limits, capacity limits, and storage costs), and discuss the outcomes from the two test scenarios.

### Summary of `price_contract` Functionality:

The `price_contract` function effectively simulates a natural gas storage contract, calculating its net value over time. It processes injection and withdrawal events chronologically, meticulously tracking the storage level. Key functionalities include:

1.  **Event Processing**: It combines and sorts all injection and withdrawal events by date, ensuring they are processed in chronological order.
2.  **Storage Level Management**: It maintains a running `storage_level`, which increases with injections and decreases with withdrawals.
3.  **Rate Limits**: Both `injection_rate` and `withdrawal_rate` parameters are respected, preventing transactions from exceeding the maximum allowable volume per event.
4.  **Capacity Limits**: The `max_volume` parameter ensures that the storage level never exceeds the physical capacity, and injections are capped if they would cause an overflow.
5.  **Storage Cost Calculation**: Daily storage costs (`storage_cost_per_unit_per_day`) are accurately applied for the duration gas is held in storage between events, impacting the total net value.
6.  **Value Calculation**: It accrues costs for injections (negative value) and revenues for withdrawals (positive value), along with the daily storage costs, to arrive at a total net value for the contract.

### Test Results:

**Scenario 1: Basic Operations**

*   **Parameters**: Sufficient `injection_rate` (500), `withdrawal_rate` (600), `max_volume` (1000), and a low `storage_cost` (0.01).
*   **Events**: Multiple injections and withdrawals that largely stay within capacity and rate limits.
*   **Net Value**: `$-1085.00`
*   **Interpretation**: This scenario demonstrated the basic functionality of the contract pricing, where the total cost of acquiring gas and storing it outweighed the revenue generated from selling it. This often happens if sell prices are not significantly higher than purchase prices, or if storage duration is long.

**Scenario 2: Capacity & Rate Limits, Longer Storage**

*   **Parameters**: Lower `injection_rate` (200), `withdrawal_rate` (250), smaller `max_volume` (500), and a higher `storage_cost` (0.02).
*   **Events**: Events designed to test limits, such as an initial injection attempting to exceed `max_volume` and `injection_rate`, and withdrawals attempting to exceed `withdrawal_rate` or available storage.
*   **Net Value**: `$870.00`
*   **Interpretation**: In this scenario, despite facing tighter constraints and higher storage costs, the contract yielded a positive net value. This indicates that the strategy of buying at lower prices (e.g., 4.0, 4.2, 4.5) and selling at significantly higher prices (e.g., 6.0, 6.5, 7.0) effectively leveraged the storage, even with rate and capacity limitations, resulting in profitability.

## Final Task


Summarize the developed `price_contract` function and present the results from the sample tests, explaining how the model accounts for various pricing factors.


## Insights

### Q&A
The `price_contract` function was successfully developed to simulate a natural gas storage contract. It calculates the net value by processing injection and withdrawal events chronologically, tracking storage levels, applying rate limits (both `injection_rate` and `withdrawal_rate`), respecting maximum storage capacity (`max_volume`), and deducting daily storage costs (`storage_cost_per_unit_per_day`). The function accrues costs for injections and revenues for withdrawals to determine the final net value.

When tested with sample inputs:
*   **Scenario 1 (Basic Operations):** With sufficient capacity and rates, and a low storage cost, the contract resulted in a net value of -\$1085.00. This indicates that the costs of acquiring and storing gas outweighed the revenues from selling it, likely due to insufficient price differentials or storage duration.
*   **Scenario 2 (Capacity & Rate Limits, Longer Storage):** Despite tighter constraints (lower rates, smaller capacity) and higher storage costs, this scenario yielded a positive net value of \$870.00. This profitability was driven by a strategy of buying gas at lower prices (\$4.0, \$4.2, \$4.5) and selling it at significantly higher prices (\$6.0, \$6.5, \$7.0), effectively leveraging the storage even with operational limitations.

### Data Analysis Key Findings
*   The `price_contract` function accurately models the net value of a natural gas storage contract, incorporating injection/withdrawal events, rates, maximum storage capacity, and daily storage costs.
*   In a basic operational scenario (Scenario 1), the contract resulted in a net loss of \$1085.00, suggesting that the pricing strategy or market conditions were not favorable enough to overcome storage and purchase costs.
*   Conversely, in a more constrained scenario (Scenario 2) with lower rates, smaller capacity, and higher storage costs, the contract generated a net profit of \$870.00, demonstrating the model's ability to capture profitability when there are significant price differentials between purchase and sale events.
*   The function correctly applies limits, ensuring that actual injection volumes do not exceed the `injection_rate` or the remaining `max_volume`, and actual withdrawal volumes do not exceed the `withdrawal_rate` or the `storage_level`.

### Insights or Next Steps
*   The `price_contract` function provides a robust framework for evaluating natural gas storage contract profitability under various operational and market conditions. Future analysis could involve optimizing storage strategies (e.g., optimal timing/volume for injection/withdrawal) to maximize net value.
*   To make Scenario 1 profitable, further investigation into the spread between injection and withdrawal prices would be beneficial, or a reduction in storage costs/duration. Sensitivity analysis on `storage_cost_per_unit_per_day` and event prices could provide insights into critical profitability thresholds.
